## Using SG-Net encoded features for RAG

### 1. Room features (from RAM tags)

In [ ]:
import os
import torch

scene_dir = '/data2/ScanNetRag/val/scene0025_00c'
with open(scene_dir + '/tags.txt', 'r') as f:
    tags = f.readlines()[0]
    f.close()

    print('The scene has tags: {}'.format(tags))

### 2. Object features
In SG-Reg, we encode multiple modality of a semantic object:
- Local topology feature $\mathbf{x}$.
- Shape feature $\mathbf{f}$.
- Dense point feature $\mathbf{z}$.

<!-- <p align="center">
    <img src="explicit_sg.png" width="520"/>
</p> -->

We encode these features on ScanNet dataset. Please use the following code to load the encoded features.



In [ ]:

feature_dict = torch.load(scene_dir + '/features.pth')
idxs = feature_dict['instances'] # (N,)
x = feature_dict['x'] # (N,d)
f = feature_dict['f'] # (N,ds)
labels = feature_dict['labels'] # (N,)
N = x.shape[0]

print('Load {} object features from {}'.format(N, scene_dir))


### 3. Dense point cloud with instance labels

In [ ]:
xyzi = torch.load(scene_dir + '/xyzi.pth') # (P,4)
xyzi = xyzi.to(idxs.device)
P = xyzi.shape[0]
points = xyzi[:,:3] # (P,3)
instances = xyzi[:,3].to(torch.int32) # (P,)
print('Load {} points from {}'.format(P, scene_dir))
print('Check the point cloud of each object:')
for i, idx in enumerate(idxs):
    masks = instances == idx
    print('  object {} ({}) has {} points'.format(idx, labels[i], masks.sum()))


### Visualize a scene (have not tested)

In [ ]:
import open3d as o3d 
from open3d.web_visualizer import draw

explicit_scene_folder = '/data2/ScanNetGraph/val/scene0064_00c'

instance_map_dir = os.path.join(explicit_scene_folder, '0002.ply')
pcd = o3d.io.read_point_cloud(instance_map_dir)
print('Load {} points from {}'.format(len(pcd.points), instance_map_dir))
